## IMPORTS

In [ ]:
!pip install datasets transformers 
# ==4.28.0
!pip install scikit-multilearn
!pip install accelerate -U
!pip install tensorflow
%env SM_FRAMEWORK=tensorflow.keras
!pip install keras

In [4]:
# !pip install nltk
!pip install torch

  Using cached torch-2.0.1-cp310-cp310-manylinux1_x86_64.whl (619.9 MB)
  Obtaining dependency information for filelock from https://files.pythonhosted.org/packages/00/45/ec3407adf6f6b5bf867a4462b2b0af27597a26bd3cd6e2534cb6ab029938/filelock-3.12.2-py3-none-any.whl.metadata
  Using cached filelock-3.12.2-py3-none-any.whl.metadata (2.7 kB)
  Using cached sympy-1.12-py3-none-any.whl (5.7 MB)
  Using cached networkx-3.1-py3-none-any.whl (2.1 MB)
  Using cached nvidia_cuda_nvrtc_cu11-11.7.99-2-py3-none-manylinux1_x86_64.whl (21.0 MB)
  Using cached nvidia_cuda_runtime_cu11-11.7.99-py3-none-manylinux1_x86_64.whl (849 kB)
  Using cached nvidia_cuda_cupti_cu11-11.7.101-py3-none-manylinux1_x86_64.whl (11.8 MB)
  Using cached nvidia_cudnn_cu11-8.5.0.96-2-py3-none-manylinux1_x86_64.whl (557.1 MB)
  Using cached nvidia_cublas_cu11-11.10.3.66-py3-none-manylinux1_x86_64.whl (317.1 MB)
  Using cached nvidia_cufft_cu11-10.9.0.58-py3-none-manylinux1_x86_64.whl (168.4 MB)
  Using cached nvidia_curand_cu

In [1]:
import pickle
import pandas as pd
import numpy as np
import nltk
import torch
import string
import os
import tensorflow as tf
import time
from time import sleep
from string import punctuation
from sklearn import metrics
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, TFBertModel
from datasets import load_dataset, load_from_disk, DatasetDict, Dataset
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
from tensorflow.keras.preprocessing import image
from PIL import Image

2023-08-18 22:12:42.503923: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-18 22:12:44.138226: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/jessica/.pyenv/versions/3.10.6/envs/movie_genre_prediction/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Data

In [27]:
long_df = pd.read_csv('long_sypnosis.csv').drop(columns = "Unnamed: 0")

In [31]:
corrupted_list = []

for img in tqdm(range(long_df.shape[0])):
    try:
        images_path = f"/home/jessica/code/chooj202/movie_genre_prediction/notebooks/merged/{long_df['imdb_id'][img]}.jpg"
        image = Image.open(images_path)
    except:
        corrupted_list.append(long_df['imdb_id'][img])

100%|█████████████████████████████████████████████████████████████████████████████| 4624/4624 [00:02<00:00, 1812.31it/s]


In [33]:
len(corrupted_list)

681

In [34]:
long_df = long_df.loc[~long_df['imdb_id'].isin(corrupted_list)]

In [44]:
long_df = long_df.reset_index(drop=True)

## Preprocess Data
images -> resize and scale
text -> word_ids?

In [46]:
long_df["genre"] = (
    long_df["genre"]
    .apply(eval)
    .apply(lambda x: [genre.strip() for genre in x])
    )

In [48]:
long_df.head()

,movie,imdb_id,genre,image_url,sypnosis
0,Ramayana: The Legend of Prince Rama,tt0259534,"[Animation, Action, Adventure]",http://img.omdbapi.com/?i=tt0259534&h=600&apik...,An anime adaptation of the Hindu epic the Rama...
1,The Dark Knight,tt0468569,"[Action, Crime, Drama]",http://img.omdbapi.com/?i=tt0468569&h=600&apik...,When the menace known as the Joker wreaks havo...
2,The Lord of the Rings: The Return of the King,tt0167260,"[Action, Adventure, Drama]",http://img.omdbapi.com/?i=tt0167260&h=600&apik...,Gandalf and Aragorn lead the World of Men agai...
3,Spider-Man: Across the Spider-Verse,tt9362722,"[Animation, Action, Adventure]",http://img.omdbapi.com/?i=tt9362722&h=600&apik...,"Miles Morales catapults across the Multiverse,..."
4,The Message,tt0075143,"[Action, Adventure, Biography]",http://img.omdbapi.com/?i=tt0075143&h=600&apik...,"The story of prophet ""Muhammad"" and the delive..."


In [53]:
def cleaning(sentence):

    # Basic cleaning
    sentence = sentence.strip() ## remove whitespaces
    sentence = sentence.lower() ## lowercase
    sentence = ''.join(char for char in sentence if not char.isdigit()) ## remove numbers

    # Advanced cleaning
    for punctuation in string.punctuation:
        sentence = sentence.replace(punctuation, '') ## remove punctuation

    return sentence

In [54]:
long_df['sypnosis'] = long_df['sypnosis'].apply(cleaning)

In [56]:
long_df.head()

,movie,imdb_id,genre,image_url,sypnosis
0,Ramayana: The Legend of Prince Rama,tt0259534,"[Animation, Action, Adventure]",http://img.omdbapi.com/?i=tt0259534&h=600&apik...,an anime adaptation of the hindu epic the rama...
1,The Dark Knight,tt0468569,"[Action, Crime, Drama]",http://img.omdbapi.com/?i=tt0468569&h=600&apik...,when the menace known as the joker wreaks havo...
2,The Lord of the Rings: The Return of the King,tt0167260,"[Action, Adventure, Drama]",http://img.omdbapi.com/?i=tt0167260&h=600&apik...,gandalf and aragorn lead the world of men agai...
3,Spider-Man: Across the Spider-Verse,tt9362722,"[Animation, Action, Adventure]",http://img.omdbapi.com/?i=tt9362722&h=600&apik...,miles morales catapults across the multiverse ...
4,The Message,tt0075143,"[Action, Adventure, Biography]",http://img.omdbapi.com/?i=tt0075143&h=600&apik...,the story of prophet muhammad and the delivery...


In [57]:
from skmultilearn.model_selection import iterative_train_test_split

mlb = MultiLabelBinarizer()

def balanced_split(df, mlb, test_size=0.5):
    ind = np.expand_dims(np.arange(len(df)), axis=1)
    mlb.fit_transform(df["genre"])
    labels = mlb.transform(df["genre"])
    ind_train, _, ind_test, _ = iterative_train_test_split(
        ind, labels, test_size
    )
    return df.iloc[ind_train[:, 0]], df.iloc[ind_test[:, 0]]

In [58]:
df_train, df_test = balanced_split(long_df, mlb, test_size=0.2)
# df_val, df_test = balanced_split(df_tmp, test_size=0.5)

In [66]:
df_train.shape

(3136, 5)

In [64]:
df_train = df_train.reset_index(drop = True)
df_test = df_test.reset_index(drop = True)

In [69]:
df_train.to_csv('stack_train.csv')
df_test.to_csv('stack_test.csv')

In [65]:
df_test.shape

(807, 5)

In [70]:
train_imgs = []

for img in tqdm(range(df_train.shape[0])):
    images_path = f"/home/jessica/code/chooj202/movie_genre_prediction/notebooks/merged/{df_train['imdb_id'][img]}.jpg"
    image = Image.open(images_path)
    image = image.resize((224, 224))
    train_imgs.append(np.array(image))
    time.sleep(1)
    np.save('train_img', train_imgs)

100%|█████████████████████████████████████████████████████████████████████████████| 3136/3136 [1:07:23<00:00,  1.29s/it]


In [71]:
train_imgs = np.array(train_imgs)
np.save('train_imgs', train_imgs)

In [72]:
test_imgs = []

for img in tqdm(range(df_test.shape[0])):
    images_path = f"/home/jessica/code/chooj202/movie_genre_prediction/notebooks/merged/{df_test['imdb_id'][img]}.jpg"
    image = Image.open(images_path)
    image = image.resize((224, 224))
    test_imgs.append(np.array(image))
    time.sleep(1)

test_imgs = np.array(test_imgs)
np.save('test_imgs', test_imgs)

100%|█████████████████████████████████████████████████████████████████████████████████| 807/807 [13:37<00:00,  1.01s/it]


In [75]:
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(df_train['genre'])

#transform target variable
y_train = multilabel_binarizer.transform(df_train['genre'])
genre_names = multilabel_binarizer.classes_

# Adding
for i in range(len(genre_names)):
    df_train[f"{genre_names[i]}"] = y_train[:,i]

In [77]:
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(df_test['genre'])

#transform target variable
y_test = multilabel_binarizer.transform(df_test['genre'])
genre_names = multilabel_binarizer.classes_

# Adding
for i in range(len(genre_names)):
    df_test[f"{genre_names[i]}"] = y_test[:,i]

In [79]:
np.save('y_train_stack', y_train)
np.save('y_test_stack', y_test)

In [91]:
df_train.head()

,movie,imdb_id,genre,image_url,sypnosis,Action,Adventure,Animation,Biography,Comedy,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,Ramayana: The Legend of Prince Rama,tt0259534,"[Animation, Action, Adventure]",http://img.omdbapi.com/?i=tt0259534&h=600&apik...,an anime adaptation of the hindu epic the rama...,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,The Dark Knight,tt0468569,"[Action, Crime, Drama]",http://img.omdbapi.com/?i=tt0468569&h=600&apik...,when the menace known as the joker wreaks havo...,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Spider-Man: Across the Spider-Verse,tt9362722,"[Animation, Action, Adventure]",http://img.omdbapi.com/?i=tt9362722&h=600&apik...,miles morales catapults across the multiverse ...,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Inception,tt1375666,"[Action, Adventure, Sci-Fi]",http://img.omdbapi.com/?i=tt1375666&h=600&apik...,a thief who steals corporate secrets through t...,1,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,The Lord of the Rings: The Two Towers,tt0167261,"[Action, Adventure, Drama]",http://img.omdbapi.com/?i=tt0167261&h=600&apik...,while frodo and sam edge closer to mordor with...,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [161]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

train_text = df_train['sypnosis'].to_list()

In [163]:
len(train_text)

3136

In [164]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

test_text = df_test['sypnosis'].to_list()

In [111]:
len(test_text)

512

In [ ]:
train_encodings = tokenizer(str(train_text), truncation=True, padding=True, max_length=128, return_tensors="np")
test_encodings = tokenizer(str(test_text), truncation=True, padding=True, max_length=128, return_tensors="np")

train_encodings = dict(train_encodings)
test_encodings = dict(valid_encodings)

# model.fit(train_encodings, tf.convert_to_tensor(y_train), epochs=3)

In [126]:
train_encodings

{'input_ids': array([[  101,  1031,  1005,  2019,  8750,  6789,  1997,  1996,  7560,
          8680,  1996, 14115, 16811,  2073,  2935,  8223,  4337,  2015,
          1996, 10433,  2332, 10958, 27313,  1996, 10392,  2466,  1997,
         14115,  1037,  2402,  3159,  2040,  2038,  2042, 21319,  2000,
          1996,  3224,  2011,  2010, 26959,  2002,  2003,  2104,  1996,
          3860,  1997,  2010,  2564,  4133,  2050,  1998,  2010,  2567,
          2474,  5705, 13890,  2043,  1037,  3928,  5698,  2332, 10958,
          6212, 19935, 14194,  3215,  4133,  2050, 14115, 13416,  2046,
          4000,  1998, 14038,  2021,  2002, 12237,  2844,  1998,  9590,
          2002,  2442,  2954,  6750,  7942,  2491,  2010, 14038,  1998,
          2954,  2127,  2010,  2564,  2003,  2489,  1517, 14115,  1996,
          8987,  3159,  1997,  1037,  7677, 16425,  3148, 14115,  2003,
         14146,  2005,  7426,  2086,  2000,  1996,  3224,  2010,  2564,
          4133,  2050,  1998,  2567,  2474,  5705, 

In [129]:
with open("test_stack", "wb") as fp:   #Pickling
    pickle.dump(valid_encodings, fp)

# >>> with open("test", "rb") as fp:   # Unpickling
# ...   b = pickle.load(fp)
# ... 
# >>> b
# [1, 2, 3, 4]

In [130]:
with open("train_stack", "wb") as fp:   #Pickling
    pickle.dump(train_encodings, fp)

# >>> with open("test", "rb") as fp:   # Unpickling
# ...   b = pickle.load(fp)
# ... 
# >>> b
# [1, 2, 3, 4]

In [110]:
test_text = test_text['input_ids']

In [106]:
train_text = train_text['input_ids']

In [149]:
# load the dataset and split into train/test sets
# REMEMBER TO DROP COLUMNS!!
X_train_img = train_imgs
X_train_text = train_encodings['input_ids']
y_train = y_train
X_test_img = test_imgs
X_test_text = valid_encodings['input_ids']
y_test = y_test

128

## CNN Model

In [132]:
# define the CNN-based image feature extractor
def build_image_model():
 img_model = tf.keras.Sequential([
 tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
 tf.keras.layers.MaxPooling2D((2, 2)),
 tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
 tf.keras.layers.MaxPooling2D((2, 2)),
 tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
 tf.keras.layers.MaxPooling2D((2, 2)),
 tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
 tf.keras.layers.MaxPooling2D((2, 2)),
 tf.keras.layers.Flatten(),
 tf.keras.layers.Dense(512, activation='relu')
 ])
 return img_model

## BERT Model

In [133]:
# define the BERT-based text feature extractor
def build_text_model():
 bert_model = TFBertModel.from_pretrained('bert-base-uncased')
 inputs = tf.keras.layers.Input(shape=(None,), dtype=tf.int32, name='input_word_ids')
 outputs = bert_model(inputs)[1]
 text_model = tf.keras.Model(inputs=inputs, outputs=outputs)
 return text_model

## Multimodal Model

In [134]:
# define the multimodal document classification model
def build_multimodal_model(num_classes):
 img_model = build_image_model()
 text_model = build_text_model()
 img_input = tf.keras.layers.Input(shape=(224, 224, 3), name='img_input')
 text_input = tf.keras.layers.Input(shape=(None,), dtype=tf.int32, name='text_input')
 img_features = img_model(img_input)
 text_features = text_model(text_input)
 concat_features = tf.keras.layers.concatenate([img_features, text_features])
 x = tf.keras.layers.Dense(512, activation='relu')(concat_features)
 x = tf.keras.layers.Dense(num_classes, activation='sigmoid')(x)
 multimodal_model = tf.keras.Model(inputs=[img_input, text_input], outputs=x)
 return multimodal_model

In [135]:
# build the multimodal model
num_classes = 21
multimodal_model = build_multimodal_model(num_classes)
multimodal_model.summary()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 img_input (InputLayer)      [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 text_input (InputLayer)     [(None, None)]               0         []                            
                                                                                                  
 sequential_1 (Sequential)   (None, 512)                  9678528   ['img_input[0][0]']           
                                                                                                  
 model_2 (Functional)        (None, 768)                  1094822   ['text_input[0][0]']          
                                                          40                                

In [153]:
# compile the model and train on the train set
multimodal_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'f1', 'roc_auc'])

multimodal_model.fit([tf.convert_to_tensor(X_train_img), tf.convert_to_tensor(X_train_text)], tf.convert_to_tensor(y_train), epochs=2, batch_size=32, validation_data=([tf.convert_to_tensor(X_test_img), tf.convert_to_tensor(X_test_text)], tf.convert_to_tensor(y_test)))

ValueError: Data cardinality is ambiguous:
  x sizes: 3136, 1
  y sizes: 3136
Make sure all arrays contain the same number of samples.